# Ноутбук с описанием шагов по созданию DeepFake видео.

Для обучение модели, конвертирования и тд мы будем использовать библиотеку faceswap.

### Сначала подготоим виртуальную машину для работы: 
Кликните на вкладку "Среда выполнения", нажниме на "Сменить среду выполнения" и удостоверьтесь что выбран GPU.

PS. Бесплатная версия коллаба очень часто обрывает сессию, поэтому если вы хотите быстро получить результат и сэкономить нервы, то возьмите PRO версию. С ней выделяют более мощную видеокарты и обрывают сессию раз в день.


In [ ]:
# Клонируем репозиторий с библиотекой c помощью которой будем обучать модель.
!rm -rf faceswap
!git clone https://github.com/kudddy/faceswap.git

Cloning into 'faceswap'...
remote: Enumerating objects: 9743, done.
remote: Counting objects: 100% (313/313), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 9743 (delta 184), reused 263 (delta 169), pack-reused 9430
Receiving objects: 100% (9743/9743), 195.45 MiB | 27.43 MiB/s, done.
Resolving deltas: 100% (6799/6799), done.


In [ ]:
# на виртуальной машине из под коробки установлена версия tf, которая не совместима с faceswap. Удалим ее
pip uninstall tensorflow

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.6.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.6.0


In [ ]:
# установим список зависимостей. без них faceswap не заработает
!pip install -r faceswap/requirements_nvidia.txt

  Cloning https://github.com/deepfakes/nvidia-ml-py3.git to /tmp/pip-req-build-lm1i8a8x
  Running command git clone -q https://github.com/deepfakes/nvidia-ml-py3.git /tmp/pip-req-build-lm1i8a8x
Ignoring pywin32: markers 'sys_platform == "win32"' don't match your environment
Ignoring pynvx: markers 'sys_platform == "darwin"' don't match your environment
     |████████████████████████████████| 296 kB 15.1 MB/s 
     |████████████████████████████████| 155 kB 62.3 MB/s 
     |████████████████████████████████| 3.3 MB 51.8 MB/s 
     |████████████████████████████████| 26.9 MB 1.5 MB/s 
     |████████████████████████████████| 394.5 MB 545 bytes/s 
     |████████████████████████████████| 2.9 MB 50.7 MB/s 
     |████████████████████████████████| 462 kB 56.2 MB/s 
     |████████████████████████████████| 3.8 MB 50.5 MB/s 
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.1-py3-none-any.whl size=19264 sha256=22a8a95152ac94a7e78fcbd60a15f7bcac9611ae01dc1c2b590bd8266c175d7d
  Stored in

In [ ]:
# При инициализации новой сессии все данные с виртуальной машины удаляются,  поэтому нам требуется постоянное хранилище.
# Это процедура крайне необходима потому что каждый раз заливать данные для обучения слишком долго и нам где то нужно хранить состояния модели.
# Сохранять состояние модели требуется для продолжения обучения не с самого начала, а с момента когда оборвалась сессия.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# удостоверимся что google выделила нам виртуальную машину.
!nvidia-smi

Sun Oct 17 13:05:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from os import path, mkdir, listdir
from tqdm import tqdm

In [ ]:
folders = ['face_swap_video', 'faces_swap_photos', 'face_swap_align']
def generate_folders_for_exp(test_number: int):
    main_path = '/content/drive/MyDrive/faceswap'
    exp_path = path.join(main_path, f'test_{test_number}')
    if not path.exists(exp_path):
        mkdir(exp_path)
    for folder in folders:
        folder_path = path.join(exp_path, folder)
        if not path.exists(folder_path):
            mkdir(folder_path)
        for name in ['nasty', 'veronika']:
            name_path = path.join(folder_path, name)
            if not path.exists(name_path):
                mkdir(name_path)
            for number in ['one', 'two', 'there', 'four', 'five', 'six']:
                number_path = path.join(name_path, number)
                if not path.exists(number_path):
                    mkdir(number_path)

In [ ]:
generate_folders_for_exp(4)

In [ ]:
def lol(test_number:int):
    image_dir_vm = '/content/images/'
    if not path.exists(image_dir_vm):
        mkdir(image_dir_vm)
    for face in ['A', 'B']:
        face_dir = path.join(image_dir_vm, face)
        if not path.exists(face_dir):
            mkdir(face_dir)
    main_path = '/content/drive/MyDrive/faceswap'
    exp_path = path.join(main_path, f'test_{test_number}')

    for number in tqdm(['one', 'two', 'there', 'four', 'five', 'six']):
      paths = path.join(exp_path, 'faces_swap_photos', 'nasty', number)
      dir = path.join(image_dir_vm, 'A')
      !cp -a '{paths}'/. '{dir}'
    
    for number in tqdm(['one', 'two', 'there', 'four', 'five', 'six']):
      paths = path.join(exp_path, 'faces_swap_photos', 'veronika', number)
      dir = path.join(image_dir_vm, 'B')
      !cp -a '{paths}'/. '{dir}'
  

# Процесс извлечения лиц из исходников видео.

Для обучения модели требуется большой массив данных. Желательно HD качества, если есть Blu-ray, то вы только выиграете от этого. Вообщем, чем лучше качество исходников тем лучше. 

Но реальность такова что достать видео такого качества достаточно сложно, поэтому можно скачать исходники из youtube воспользовавших одним из многочисленных сервисов(я использовал https://ru.savefrom.net). Youtube, к сожалению, сильно сжимает качество, но тем не менее даже с такими данным можно получить более-менее качественный deepfake.

In [1]:
# Что делаем? берем видео, видео разбиваем на кадры, в каждом кадре фиксируем лицо, сохраняем фото в папку. Так же алгоритм сгенерирует файл fsa, где хранятся координаты лица в кадре.
# Запускаем алгоритм с более менее оптимальными параметрами.
test = 4
actress = 'veronika'
for video_number in ['one', 'two', 'there', 'four', 'five', 'six']:
  input_video = f'/content/drive/MyDrive/faceswap/test_{test}/face_swap_video/{actress}/{video_number}/{actress}_{video_number}_norm.mp4'
  output_photo = f'/content/drive/MyDrive/faceswap/test_{test}/faces_swap_photos/{actress}/B'
  aligments_file = f'/content/drive/MyDrive/faceswap/test_{test}/face_swap_align/{actress}/{video_number}/aligments_2.fsa'
  !python3 faceswap/faceswap.py extract \
    -i '{input_video}' \
    -o '{output_photo}' \
    -al '{aligments_file}' \
    -D 's3fd' \
    -A 'fan' \
    -nm 'hist' \
    -een 10


python3: can't open file 'faceswap/faceswap.py': [Errno 2] No such file or directory
python3: can't open file 'faceswap/faceswap.py': [Errno 2] No such file or directory
python3: can't open file 'faceswap/faceswap.py': [Errno 2] No such file or directory
python3: can't open file 'faceswap/faceswap.py': [Errno 2] No such file or directory
python3: can't open file 'faceswap/faceswap.py': [Errno 2] No such file or directory
python3: can't open file 'faceswap/faceswap.py': [Errno 2] No such file or directory


# Процесс обучения модели.

По времени процесс обычно занимает около 5-6 дней. Но сроки можно смещать в право/влево в зависимости от качества, которое вам нужно. 

Есть несколько гиперпараметров на которые стоит обратить внимание: 
ps еще про px рассказать
- trainer_type - я ставлю phaze-a, но она очень требовательна к производительности, поэтому можно начать с dfaker
- save_model_every - параметр который говорит алгоритм когда сохранять веса модель. Важно задать его чтобы не потерять веса и в случае разрыва сессии начать с конечного момента.

In [5]:
# у collab есть забавный баг, который не позволяет получить доступ к папке, где больше 10000 тысяч объектов. 
# но это можно обойти. Запросы к gdrive кэшируются,  поэтому если запустить,  например, команду ниже, то на третий раз мы получим доступ к папке.
# Это важно так как скрипт для обучения модели должен получить доступ к данным.

# жмякаем до момента, когда ошибка уйдет
len(listdir('/content/drive/MyDrive/faceswap/test_4/faces_swap_photos/nasty/A'))

In [ ]:
"НУЖНО НЕ ЗАБЫТЬ ПОМЕНЯТЬ НАСТРОЙКИ ТРЕЙНА"

In [ ]:
# начинаем процесс обучения
num_iterations = "1000000"
save_every = "360"
save_model_every = "25000"
batch_num = "8"
num_gpus = "1"
# тип используемой модели. помимо phaze-a можно использовать dfaker,  но она даст менее качественный вариант.
trainer_type = "phaze-a"

# тут мы сохраняем модель
model_dir = "/content/drive/MyDrive/faceswap/test_6/model"
# путь к папке с извлеченными фотографиями из видео  лица 1
face_pic_file_a = "/content/drive/MyDrive/faceswap/test_4/faces_swap_photos/nasty/A"
# путь к папке с извлеченными фотографиями из видео  лица 2
face_pic_file_b = "/content/drive/MyDrive/faceswap/test_4/faces_swap_photos/veronika/B"
# сюда мы сохраняем картинку, где можно наглядно посмотреть на преобразовательную возможность модели.
timelapse_dir = "/content/drive/MyDrive/faceswap/test_6/timelapse"
#set variables end


### начнаем процесс обучения.
!python3 faceswap/faceswap.py train \
  -A '{face_pic_file_a}' \
  -B '{face_pic_file_b}' \
  -m '{model_dir}' \
  -t '{trainer_type}' \
  -bs '{batch_num}' \
  -it '{num_iterations}' \
  -s '{save_every}' \
  -ss '{save_model_every}' \
  -tia '{face_pic_file_a}' \
  -tib '{face_pic_file_b}' \
  -to '{timelapse_dir}' \
  -w \
  -nw

### после 500к - 700к модель должна выдавать приемлемый результат, но для 

First time configuration. Please select the required backend
1: AMD, 2: CPU, 3: NVIDIA: 3
Faceswap config written to: /content/faceswap/config/.faceswap
Setting Faceswap backend to NVIDIA
10/11/2021 08:23:04 INFO     Log level set to: INFO
10/11/2021 08:23:07 INFO     Model A Directory: '/content/drive/MyDrive/faceswap/test_4/faces_swap_photos/nasty/A' (29375 images)
10/11/2021 08:23:07 INFO     Model B Directory: '/content/drive/MyDrive/faceswap/test_4/faces_swap_photos/veronika/B' (27731 images)
10/11/2021 08:23:07 INFO     Training data directory: /content/drive/MyDrive/faceswap/test_6/model
10/11/2021 08:23:07 INFO     ===================================================
10/11/2021 08:23:07 INFO       Starting
10/11/2021 08:23:07 INFO       Press 'ENTER' to save and quit
10/11/2021 08:23:07 INFO       Press 'S' to save model weights immediately
10/11/2021 08:23:07 INFO     ===================================================
10/11/2021 08:23:08 INFO     Loading data, this may take a 

In [ ]:
!python3 faceswap/faceswap.py extract -i '/content/drive/MyDrive/faceswap/test_one/face_swap_video/nasty/nasty_for_convert.mp4' -o '/content/drive/MyDrive/faceswap/test_one/faces_swap_photos/nasty_for_convert' -al '/content/drive/MyDrive/faceswap/test_one/face_swap_align/nasty_for_convert' -D 's3fd' -A 'fan' -nm 'hist'

Setting Faceswap backend to NVIDIA
09/28/2021 07:53:22 INFO     Log level set to: INFO
09/28/2021 07:53:27 INFO     Output Directory: /content/drive/MyDrive/faceswap/test_one/faces_swap_photos/nasty_for_convert
09/28/2021 07:53:28 INFO     Loading Detect from S3Fd plugin...
09/28/2021 07:53:28 INFO     Downloading model: 's3fd_keras' from: https://github.com/deepfakes-models/faceswap-models/releases/download/v1.11.2/s3fd_keras_v2.zip
Downloading: 100% 11.3M/11.3M [00:00<00:00, 60.0MB/s]
09/28/2021 07:53:29 INFO     Extracting: 's3fd_keras'
Decompressing: 100% 85.8M/85.8M [00:00<00:00, 92.6MB/s]
09/28/2021 07:53:30 INFO     Loading Align from Fan plugin...
09/28/2021 07:53:30 INFO     Downloading model: 'face-alignment-network_2d4_keras' from: https://github.com/deepfakes-models/faceswap-models/releases/download/v1.13.2/face-alignment-network_2d4_keras_v2.zip
Downloading: 100% 84.7M/84.7M [00:01<00:00, 71.2MB/s]
09/28/2021 07:53:31 INFO     Extracting: 'face-alignment-network_2d4_keras'

In [ ]:
!python3 faceswap/faceswap.py convert \
  -m '/content/drive/MyDrive/faceswap/test_one/model' \
  -w 'ffmpeg' \
  -i '/content/drive/MyDrive/faceswap/test_one/face_swap_video/nasty/nasty_for_convert.mp4' \
  -o '/content/drive/MyDrive/faceswap/test_one/result' \
  -al '/content/drive/MyDrive/faceswap/test_one/face_swap_align/nasty_for_convert.fsa' 

Setting Faceswap backend to NVIDIA
09/28/2021 08:07:34 INFO     Log level set to: INFO
09/28/2021 08:07:36 INFO     Reading alignments from: '/content/drive/MyDrive/faceswap/test_one/face_swap_align/nasty_for_convert.fsa'
09/28/2021 08:07:36 INFO     Loading Writer from Ffmpeg plugin...
09/28/2021 08:07:36 INFO     Loading Model from Dfaker plugin...
09/28/2021 08:07:36 INFO     Using configuration saved in state file
09/28/2021 08:07:40 INFO     Loaded model from disk: '/content/drive/MyDrive/faceswap/test_one/model/dfaker.h5'
09/28/2021 08:07:40 INFO     Loading Mask from Box_Blend plugin...
09/28/2021 08:07:40 INFO     Loading Mask from Mask_Blend plugin...
09/28/2021 08:07:40 INFO     Loading Color from Avg_Color plugin...
09/28/2021 08:07:40 INFO     Loading Scaling from Sharpen plugin...
09/28/2021 08:07:43 INFO     Outputting to: '/content/drive/MyDrive/faceswap/test_one/result/nasty_for_convert_converted.mp4'
Converting: 100% 1722/1723 [03:52<00:00,  7.40it/s]
09/28/2021 08:11:

In [ ]:
print("dfdsf")

dfdsf


In [ ]:
!python3 faceswap/faceswap.py extract \
  -i '/content/drive/MyDrive/faceswap/test_6/video_for_convert/start2.MOV' \
  -o '/content/drive/MyDrive/faceswap/test_6/photos_for_swap/four' \
  -al '/content/drive/MyDrive/faceswap/test_6/alignments/nasty/alig5.fsa' \
  -D 's3fd' \
  -A 'fan' \
  -nm 'hist'\
  -M 'vgg-clear'

Setting Faceswap backend to NVIDIA
10/17/2021 13:29:10 INFO     Log level set to: INFO
10/17/2021 13:29:13 INFO     Output Directory: /content/drive/MyDrive/faceswap/test_6/photos_for_swap/four
10/17/2021 13:29:14 INFO     Loading Detect from S3Fd plugin...
10/17/2021 13:29:14 INFO     Loading Align from Fan plugin...
10/17/2021 13:29:14 INFO     Loading Mask from Components plugin...
10/17/2021 13:29:14 INFO     Loading Mask from Extended plugin...
10/17/2021 13:29:14 INFO     Loading Mask from Vgg_Clear plugin...
10/17/2021 13:29:14 INFO     Downloading model: 'Nirkin_300_softmax' from: https://github.com/deepfakes-models/faceswap-models/releases/download/v1.8.1/Nirkin_300_softmax_v1.zip
Downloading: 100% 440M/440M [00:07<00:00, 59.4MB/s]
10/17/2021 13:29:22 INFO     Extracting: 'Nirkin_300_softmax'
Decompressing: 100% 440M/440M [00:03<00:00, 132MB/s]
10/17/2021 13:29:26 INFO     Starting, this may take a while...
10/17/2021 13:29:26 INFO     Initializing S3FD (Detect)...
10/17/2021 